In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# test datainput

In [ ]:

def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
        t = example[name]
        if t.dtype == tf.int64:
            t = tf.cast(t, tf.int32)
        example[name] = t
    return example

In [ ]:
batch_size = 1
seq_length = 1025
is_training = True
num_cpu_threads = 4
evaluate_for_fixed_number_of_steps=True
input_files = ['./dataset/train_data/train.tfrecord']
# input_files = ['./temp.tfrecord']

name_to_features = {
    "input_ids": tf.FixedLenFeature([seq_length+1], tf.int64),
}
d = tf.data.Dataset.from_tensor_slices(tf.constant(input_files))
d = d.repeat()
d = d.shuffle(buffer_size=len(input_files))

# `cycle_length` is the number of parallel files that get read.
cycle_length = min(num_cpu_threads, len(input_files))

# `sloppy` mode means that the interleaving is not exact. This adds
# even more randomness to the training pipeline.
d = d.apply(
    tf.data.experimental.parallel_interleave(
        tf.data.TFRecordDataset,
        sloppy=is_training,
        cycle_length=cycle_length))
d = d.shuffle(buffer_size=100)

for _d in d.take(1):
    print(_d)
d = d.apply(
tf.data.experimental.map_and_batch(
    lambda record: _decode_record(record, name_to_features),
    batch_size=batch_size,
    num_parallel_batches=num_cpu_threads,
    drop_remainder=True))
for _d in d.take(1):
    print(_d)

In [ ]:
type(d)

In [ ]:
with tf.Session() as sess:
    print(sess.run(d.make_one_shot_iterator().get_next()))
    d = d.apply(
    tf.data.experimental.map_and_batch(
        lambda record: _decode_record(record, name_to_features),
        batch_size=batch_size,
        num_parallel_batches=num_cpu_threads,
        drop_remainder=True))
    print(d.make_one_shot_iterator().get_next()))

In [ ]:
import collections

def create_int_feature(values):
    feature = tf.train.Feature(
        int64_list=tf.train.Int64List(value=list(values)))
    return feature

ids = [
    [101, 202, 10, 40], 
    [201, 202, 10, 40],
    [301, 202, 10, 40]
]

for _id in ids:
    features = collections.OrderedDict()
    features["input_ids"] = create_int_feature(_id)
    print(features)

    train_writer = tf.python_io.TFRecordWriter("temp.tfrecord", options=None)
    tf_example = tf.train.Example(
                features=tf.train.Features(feature=features))
    train_writer.write(tf_example.SerializeToString())
train_writer.close()

In [ ]:
reader = tf.data.TFRecordDataset('temp.tfrecord')
print(reader)

image_feature_description = {'input_ids': tf.io.FixedLenFeature([4], tf.int64)}

def _parse_image_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

datas = reader.map(_parse_image_function)
print(datas)
for d in reader.take(1):
    print(repr(d))
for d in datas.take(2):
    print(repr(d))